In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import drive
drive.mount('/content/drive')
print("Files in Glove-in-bangla:")
!ls /content/drive/My\ Drive/Research/Vector/Glove-in-bangla

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in Glove-in-bangla:
corpus.txt	       log		 saved_model	  tf_glove.py
glove300d.txt	       preprocessing.py  Siyamrupali.ttf
Glove_in_bangla.ipynb  __pycache__	 text3.txt


In [0]:
with open('/content/drive/My Drive/Research/Vector/Glove-in-bangla/corpus.txt') as f:
    singleLine = f.readline()
    listoflist = []
    words = []
    while singleLine:
        words = singleLine.split()
        listoflist.append(words)
        singleLine = f.readline()

In [5]:
%cd /content/drive/My\ Drive/Research/Vector/Glove-in-bangla
!ls
import tf_glove

/content/drive/My Drive/Research/Vector/Glove-in-bangla
corpus.txt	       log		 saved_model	  tf_glove.py
glove300d.txt	       preprocessing.py  Siyamrupali.ttf
Glove_in_bangla.ipynb  __pycache__	 text3.txt


# Instantiating the model

To create a new GloVe model, simply call `tf_glove.GloVeModel()`:

In [0]:
model = tf_glove.GloVeModel(embedding_size=300, context_size=10, min_occurrences=25,
                            learning_rate=0.05, batch_size=512)

`GloVeModel()` has several parameters:
- **`embedding_size`**: the target dimensionality of the trained word representations. Typically between 50 and 300.
- **`context_size`**: how many tokens on either side of a given word to include in each context window. Can be either a tuple of two ints, indicating how many token on the left and right to include, or a single int, which will be interpreted to mean symmetric context.
- **`max_vocab_size`** *(Optional)*: the maximum size of the model's vocabulary. The model's vocabulary will be the most frequently occurring words in the corpus up to this amount. The default is 100,000.
- **`min_occurrences`** *(Optional)*: the minimum number of times a word must have appeared in the corpus to be included in the model's vocabulary. Default is 1.
- **`scaling_factor`** *(Optional)*: the alpha term in Eqn. 9 of Pennington et al.'s paper. Default is 3/4, which is the paper's recommendation
- **`cooccurrence_cap`** *(Optional)*: the x_max term in Eqn. 9 of Pennington et al.'s paper. Default is 100, which is the paper's recommendation
- **`batch_size`** *(Optional)*: the number of cooccurrences per minibatch of in training. Default is 512, which seems to work well on my machine. If training is very slow, consider playing with this.
- **`learning_rate`** *(Optional)*: the Adagrad learning rate used in training. Default is 0.05, which is the paper's recommendation

# Reading the corpus

tf_glove needs to be fit to a corpus in order to learn word representations. To do this, we'll use `GloVeModel.fit_to_corpus(corpus)`. This method expects an iterable of iterables of strings, where each string is a token, like this:

`[["this", "is", "a", "comment", "."], ["this", "is", "another", "comment", "."]]`

That was a list of lists, but any iterable of iterables of strings should work.

### Note on getting the dataset (if you want to follow along with these examples exactly)

For these examples, I'm going to use the dataset of Reddit comments described here: https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment

tf_glove is designed to work with any corpus, so there's no need to download this dataset. However, if you'd like to, that post has a link to a torrent for all of the comments as well as a link for just the comments from January 2015. Even just the January 2015 file is quite large (~5 GB).

I downloaded it and used

`$ head -n 1000000 RC_2015-01 > /path/to/RC_2015-01-1m_sample`

to get the 1 million comment sample file referenced below. You could also use 100k if you want to save some time. 1 million comments takes ~15 minutes to fit on my machine.

## The code:

In [0]:
# import re
# import nltk

# def extract_reddit_comments(path):
#     # A regex for extracting the comment body from one line of JSON (faster than parsing)
#     body_snatcher = re.compile(r"\{.*?(?<!\\)\"body(?<!\\)\":(?<!\\)\"(.*?)(?<!\\)\".*}")
#     with open(path) as file_:
#         for line in file_:
#             match = body_snatcher.match(line)
#             if match:
#                 body = match.group(1)
#                 # Ignore deleted comments
#                 if not body == '[deleted]':
#                     # Return the comment as a string (not yet tokenized)
#                     yield body
                        
# def tokenize_comment(comment_str):
#     # Use the excellent NLTK to tokenize the comment body
#     #
#     # Note that we're lower-casing the comments here. tf_glove is case-sensitive,
#     # so if you want 'You' and 'you' to be considered the same word, be sure to lower-case everything.
#     return nltk.wordpunct_tokenize(comment_str.lower())

# def reddit_comment_corpus(path):
#     # A generator that returns lists of tokens representing individual words in the comment
#     return (tokenize_comment(comment) for comment in extract_reddit_comments(path))

# # Replace the path with the path to your corpus file
# corpus = reddit_comment_corpus("/media/grady/PrimeMover/Datasets/RC_2015-01-1m_sample")

Now, to fit the model to the corpus:

In [8]:
model.fit_to_corpus(listoflist)

W0624 13:10:48.835576 140124413339520 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:71: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0624 13:10:48.844144 140124413339520 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:80: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0624 13:10:48.879179 140124413339520 deprecation.py:323] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:99: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0624 13:10:48.886825 140124413339520 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:104: The name tf.log is deprecated. Please use tf.math.log instead.

W0624 13:10:48.887632 14012441

# Training the model

GloVeModel.fit_to_corpus() builds the vocabulary and cooccurrence matrix that will be used in training, but it doesn't actually train the word representations. It's time to kick off TensorFlow and train the model for real:

In [9]:
model.train(num_epochs=50, checkpoint_dir="/content/drive/My Drive/Research/Vector/Glove-in-bangla/saved_model", log_dir="log/example", summary_batch_interval=1000)

W0624 13:11:24.194761 140124413339520 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:129: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0624 13:11:24.419865 140124413339520 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:132: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0624 13:11:24.452214 140124413339520 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:133: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



Writing TensorBoard summaries to log/example


W0624 13:11:24.821895 140124413339520 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:134: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



 Epoch 0: Model is saved Elapsed: 00:01:06.76 Steps: 30535 Loss: 128.67724609375 

 Epoch 1: Model is saved Elapsed: 00:02:11.54 Steps: 61070 Loss: 166.67669677734375 

 Epoch 2: Model is saved Elapsed: 00:03:20.30 Steps: 91605 Loss: 82.757568359375 



W0624 13:15:58.054168 140124413339520 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


 Epoch 3: Model is saved Elapsed: 00:04:33.93 Steps: 122140 Loss: 82.68478393554688 

 Epoch 4: Model is saved Elapsed: 00:05:49.15 Steps: 152675 Loss: 85.54630279541016 

 Epoch 5: Model is saved Elapsed: 00:07:04.34 Steps: 183210 Loss: 40.807098388671875 

 Epoch 6: Model is saved Elapsed: 00:08:20.08 Steps: 213745 Loss: 37.47620391845703 

 Epoch 7: Model is saved Elapsed: 00:09:34.83 Steps: 244280 Loss: 44.592185974121094 

 Epoch 8: Model is saved Elapsed: 00:10:48.76 Steps: 274815 Loss: 25.312870025634766 

 Epoch 9: Model is saved Elapsed: 00:12:02.82 Steps: 305350 Loss: 30.420089721679688 

 Epoch 10: Model is saved Elapsed: 00:13:16.14 Steps: 335885 Loss: 21.984752655029297 

 Epoch 11: Model is saved Elapsed: 00:14:23.62 Steps: 366420 Loss: 29.131816864013672 

 Epoch 12: Model is saved Elapsed: 00:15:36.91 Steps: 396955 Loss: 52.80020523071289 

 Epoch 13: Model is saved Elapsed: 00:16:49.03 Steps: 427490 Loss: 19.578214645385742 

 Epoch 14: Model is saved Elapsed: 00:17:54

`GloVeModel.train()` has a few parameters:
- **`num_epochs`**: How many passes through the cooccurrence matrix the training should make. The paper recommends at least 50 for `embedding_size` < 300, and 100 otherwise.
- **`log_dir`** *(Optional)*: The path of the directory in which to log summaries for TensorBoard and t-SNE visualizations. Default is `None`, i.e. don't log anything.
- **`summary_batch_interval`** *(Optional)*: How many minibatches between logging events for TensorBoard. Default is 1000.
- **`tsne_epoch_interval`** *(Optional)*: How many epochs (full passes through cooccurrence matrix) between outputting a t-SNE visualization of the model's embeddings for the most frequent 1000 words in the vocabulary. Default is None, i.e. don't output t-SNE visualizations during training.

# Checking out the results

Now that we've trained the model, let's look at the results.

Use `GloVeModel.embedding_for()` to get the trained embedding for a single word:

In [10]:
model.embedding_for("বাংলাদেশের")

array([ 4.49468195e-01, -2.99610347e-01,  8.90811160e-02,  6.86227530e-02,
       -3.07363927e-01, -7.31231570e-02,  2.83948362e-01, -9.87636745e-02,
       -4.07758057e-02, -8.07990655e-02, -1.25963055e-02, -4.42586094e-02,
        4.51470554e-01, -3.67618442e-01, -2.33049601e-01, -2.48302251e-01,
       -2.53085226e-01,  2.99246192e-01,  3.13928545e-01,  2.41359413e-01,
       -7.82449469e-02, -4.76090550e-01, -1.18967630e-01,  7.86635652e-02,
       -1.26175612e-01,  3.86521101e-01,  6.25950247e-02,  2.94111490e-01,
       -1.48212135e-01, -2.84734219e-01, -2.19629630e-01,  1.12609729e-01,
       -3.82431746e-01,  1.22309178e-01, -7.11721256e-02, -2.61949241e-01,
       -2.93524474e-01,  4.14523423e-01,  1.44177318e-01,  7.76035786e-02,
        2.83498794e-01,  1.97772399e-01, -4.08547744e-02,  1.42682076e-01,
       -1.92996368e-01, -7.60802031e-02,  5.00685215e-01, -4.76303101e-01,
        8.83823931e-02,  4.09157723e-02, -1.98673308e-01, -3.77011180e-01,
        1.33172721e-01,  

You can also get the model's embeddings for every word in the vocabulary like this:

`GloVeModel.embeddings` will give you a NumPy matrix where each row is the model's embedding for a single word.

To make use of this, you'll want to know what row corresponds to a particular word. You can do that with `GloVeModel.id_for_word`:

In [12]:
model.embeddings[model.id_for_word('বাংলাদেশের')]

array([ 4.49468195e-01, -2.99610347e-01,  8.90811160e-02,  6.86227530e-02,
       -3.07363927e-01, -7.31231570e-02,  2.83948362e-01, -9.87636745e-02,
       -4.07758057e-02, -8.07990655e-02, -1.25963055e-02, -4.42586094e-02,
        4.51470554e-01, -3.67618442e-01, -2.33049601e-01, -2.48302251e-01,
       -2.53085226e-01,  2.99246192e-01,  3.13928545e-01,  2.41359413e-01,
       -7.82449469e-02, -4.76090550e-01, -1.18967630e-01,  7.86635652e-02,
       -1.26175612e-01,  3.86521101e-01,  6.25950247e-02,  2.94111490e-01,
       -1.48212135e-01, -2.84734219e-01, -2.19629630e-01,  1.12609729e-01,
       -3.82431746e-01,  1.22309178e-01, -7.11721256e-02, -2.61949241e-01,
       -2.93524474e-01,  4.14523423e-01,  1.44177318e-01,  7.76035786e-02,
        2.83498794e-01,  1.97772399e-01, -4.08547744e-02,  1.42682076e-01,
       -1.92996368e-01, -7.60802031e-02,  5.00685215e-01, -4.76303101e-01,
        8.83823931e-02,  4.09157723e-02, -1.98673308e-01, -3.77011180e-01,
        1.33172721e-01,  

In [13]:
model.id_for_word('বাংলাদেশের')

88

In [30]:
embeddings = model.embeddings
embeddings[88,:2]

array([ 0.4494682 , -0.29961035], dtype=float32)

In [31]:
vocabSize = model.vocab_size
vocabSize

18689

In [32]:
vocabulary = model.words
vocabulary[88]

'বাংলাদেশের'

In [33]:
embeddings[88,:2]

array([ 0.4494682 , -0.29961035], dtype=float32)

In [0]:
    with open('/content/drive/My Drive/Research/Vector/Glove-in-bangla/glove300d.txt', 'w') as file_:
      for i in range(vocabSize):
        embed = embeddings[i, :]
        word = vocabulary[i]
        file_.write('%s %s\n' % (word, ' '.join(map(str, embed))))

And if you want to see a 2D visualization of the learned vector space, you can use `GloVeModel.generate_tsne()`:

In [36]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib
import matplotlib.font_manager
# !wget https://www.omicronlab.com/download/fonts/Siyamrupali.ttf
!cp Siyamrupali.ttf /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
!cp Siyamrupali.ttf /usr/share/fonts/truetype/

matplotlib.font_manager._rebuild()
matplotlib.rc('font', family='Siyam Rupali')
model.generate_tsne()

Output hidden; open in https://colab.research.google.com to view.

You might want to open that image in a new tab.

With no parameters, `GloVeModel.generate_tsne()` can be used interactively like in this notebook, but it also has parameters that will let you save the visualization to a file and adjust the size of the image and how many words appear:

- **`path`** *(Optional)*: The path at which to save the generated PNG image. Default is None, which only really makes sense for interactive environments.
- **`size`** *(Optional)*: A tuple of (width, height) in *inches*. (Yeah, I know right? This is inherited from matplotlib.) Default is 100 x 100.  
- **`word_count`** *(Optional)*: How many words to plot in the visualization. Default is 1000, which works fairly well for a (100 x 100) visualization.
